<a href="https://colab.research.google.com/github/nobilelucifero/colabs/blob/main/function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

General setup

In [ ]:
%%capture
!pip install openai

In [ ]:
import requests
import json

In [ ]:
from google.colab import userdata
from openai import OpenAI

In [ ]:
llm_client = OpenAI(
    api_key=userdata.get('OPENAI_KEY')
)

Set up tools

In [ ]:
def get_tov(audience, purpose):
  result = {
      "audience": audience,
      "purpose": purpose
  }

  return json.dumps(result)

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_tov",
            "description": "Get the Tone of Voice of a content source piece written by the author themselves.",
            "parameters": {
                "type": "object",
                "properties": {
                    "audience": {
                        "type": "string",
                        "description": "Defines who are the primary audiences or target demographics of the input. Each item will be a 2-3 word description.",
                    },
                    "purpose": {
                        "type": "string",
                        "description": "What's the main purpose or goal of the text?",
                    },
                },
                "required": ["audience", "purpose"],
            },
        }
    }
]

tool_choice = {"type": "function", "function": {"name": "get_tov"}}

In [ ]:
def converse(messages):
    response = llm_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto"
        # tool_choice=tool_choice
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
      messages.append(response_message)

      available_functions = {
          "get_tov": get_tov
      }

      for tool_call in tool_calls:
        print(f"Function: {tool_call.function.name}")
        print(f"Params: {tool_call.function.arguments}")

        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            audience = function_args.get("audience"),
            purpose = function_args.get("purpose"),
        )
        print(f"Tool reponse: {function_response}")

        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response
        })

        second_reponse = llm_client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = messages,
        )

        return second_reponse.choices[0].message.content


    # return response.choices[0].message;

In [ ]:
input = """
A poor Woodman was cutting down a tree near the edge of a deep pool in the forest. It was late in the day and the Woodman was tired. He had been working since sunrise and his strokes were not so sure as they had been early that morning. Thus it happened that the axe slipped and flew out of his hands into the pool.

The Woodman was in despair. The axe was all he possessed with which to make a living, and he had not money enough to buy a new one. As he stood wringing his hands and weeping, the god Mercury suddenly appeared and asked what the trouble was. The Woodman told what had happened, and straightway the kind Mercury dived into the pool. When he came up again he held a wonderful golden axe.

"Is this your axe?" Mercury asked the Woodman.

"No," answered the honest Woodman, "that is not my axe."

Mercury laid the golden axe on the bank and sprang back into the pool. This time he brought up an axe of silver, but the Woodman declared again that his axe was just an ordinary one with a wooden handle.

Mercury dived down for the third time, and when he came up again he had the very axe that had been lost.

The poor Woodman was very glad that his axe had been found and could not thank the kind god enough. Mercury was greatly pleased with the Woodman's honesty.

"I admire your honesty," he said, "and as a reward you may have all three axes, the gold and the silver as well as your own."

The happy Woodman returned to his home with his treasures, and soon the story of his good fortune was known to everybody in the village. Now there were several Woodmen in the village who believed that they could easily win the same good fortune. They hurried out into the woods, one here, one there, and hiding their axes in the bushes, pretended they had lost them. Then they wept and wailed and called on Mercury to help them.

And indeed, Mercury did appear, first to this one, then to that. To each one he showed an axe of gold, and each one eagerly claimed it to be the one he had lost. But Mercury did not give them the golden axe. Oh no! Instead he gave them each a hard whack over the head with it and sent them home. And when they returned next day to look for their own axes, they were nowhere to be found.

Honesty is the best policy.
"""

In [ ]:
result = converse(messages=[
      {
        "role": "system",
        "content": "You are a savvy copywriter for SEO, Social Media, and Blogs."
      }, {
         "role": "user",
         "content": "What's the tone of voice of this text?"
      }, {
          "role": "user",
          "content": input,
      },
])

Function: get_tov
Params: {"audience":"General","purpose":"Entertaining narrative"}
Tool reponse: {"audience": "General", "purpose": "Entertaining narrative"}


In [ ]:
print(result)

The tone of voice in the text is engaging, narrative, and moralistic with a touch of whimsicality. It captivates the reader with a story while delivering a moral lesson about honesty in a memorable and entertaining way.
